In [ ]:
# -*- coding: utf-8 -*-
"""fastapi_query_content_with_cors_fixed.ipynb

Automatically generated by Colab.
"""

# Install system dependencies
!apt-get update -qq
!apt-get install -y poppler-utils tesseract-ocr -qq

# Install Python dependencies
!pip install torch torchvision pdf2image opencv-python layoutparser pytesseract PyPDF2 fastapi uvicorn pyngrok sentence-transformers faiss-cpu psycopg2-binary --quiet
!pip install nest-asyncio --quiet

# Import libraries
import torch
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
import faiss
import psycopg2
from psycopg2.extras import execute_values
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from google.colab import drive, userdata
import asyncio

# Check if Google Drive is already mounted; mount if not
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Drive already mounted at /content/drive")

# Configuration
BASE_DIR = "/content/drive/My Drive/lifesciences"
FAISS_INDEX_PATH = Path(BASE_DIR) / "faiss_index_retrained_batch_12.index"  # Default FAISS index path

# Neon PostgreSQL connection details
DB_CONFIG = {
    "dbname": "neondb",
    "user": "neondb_owner",
    "password": "npg_DU3Vxoi6cCIu",
    "host": "ep-purple-sound-a4z952yb-pooler.us-east-1.aws.neon.tech",
    "port": "5432",
    "sslmode": "require"
}

# Global variables
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sentence_model = None
faiss_index = None
db_conn = None
db_cursor = None

# Setup database connection
def setup_db_connection():
    global db_conn, db_cursor
    if db_conn is None:
        try:
            db_conn = psycopg2.connect(**DB_CONFIG)
            db_cursor = db_conn.cursor()
            print("Connected to Neon PostgreSQL database")
        except Exception as e:
            print(f"Failed to connect to Neon PostgreSQL: {e}")
            raise

# Initialize embedding tools
def initialize_embedding_tools(faiss_index_path=FAISS_INDEX_PATH):
    global sentence_model, faiss_index
    if sentence_model is None:
        try:
            sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
            print("SentenceTransformer model loaded")
        except Exception as e:
            print(f"Error loading SentenceTransformer: {e}")
            sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
            print("Fallback to CPU for SentenceTransformer")

    if faiss_index is None:
        if faiss_index_path.exists():
            try:
                faiss_index = faiss.read_index(str(faiss_index_path))
                print(f"Loaded existing FAISS CPU index from {faiss_index_path}")
            except Exception as e:
                print(f"Error loading FAISS index: {e}")
                raise
        else:
            print(f"FAISS index not found at {faiss_index_path}. Please ensure it exists.")
            raise FileNotFoundError(f"FAISS index not found at {faiss_index_path}")

# FastAPI app
app = FastAPI(title="Content Query API")

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Replace with your Cloudflare domain
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/query_content/")
async def query_content(urs_name: str, section_name: str, k: int = 5):
    """
    Query content by urs_name, section_name, and return k nearest neighbors.
    """
    setup_db_connection()
    initialize_embedding_tools()

    try:
        # Construct query string
        separator = "|||"
        query_string = f"{urs_name}{separator}{section_name}{separator}content_type{separator}content"
        query_embedding = sentence_model.encode([query_string], convert_to_numpy=True)

        # Search FAISS index
        distances, indices = faiss_index.search(query_embedding, k)
        if indices.size == 0 or indices[0][0] == -1:
            raise HTTPException(status_code=404, detail="No matching content found")

        # Fetch content from database using faiss_index_id
        # Fetch content from database using faiss_index_id
        faiss_indices = [int(idx) for idx in indices[0]]
        print(f"Fetching content for faiss_indices: {faiss_indices}")

        # Execute query using ANY
        query = """
            SELECT cb.id, cb.source_file_run_id, cb.section_id, cb.block_number, cb.content_type, cb.content,
                  cb.coord_x1, cb.coord_y1, cb.coord_x2, cb.coord_y2, cb.created_at, cb.faiss_index_id
            FROM content_blocks cb
            WHERE cb.faiss_index_id = ANY(%s)
            ORDER BY cb.created_at DESC;
        """
        db_cursor.execute(query, (faiss_indices,))  # Note the comma to make it a single-element tuple
        results = db_cursor.fetchall()
        # Format response
        response = []
        for row in results:  # Limit to k results
            response.append({
                "id": row[0],
                "source_file_run_id": row[1],
                "section_id": row[2],
                "block_number": row[3],
                "content_type": row[4],
                "content": row[5],
                "coordinates": {"x1": row[6], "y1": row[7], "x2": row[8], "y2": row[9]},
                "created_at": row[10],
                "faiss_index_id": row[11]

            })

        return {"results": response, "count": len(response)}

    except Exception as e:
        print(f"Query error: {e}")
        raise HTTPException(status_code=500, detail=str(e))

# Function to run the FastAPI server with ngrok
def run_server():
    nest_asyncio.apply()  # Enable nested event loops in Colab
    loop = asyncio.get_event_loop()

    # Start ngrok tunnel
    ngrok.set_auth_token(userdata.get("ngrok_auth_token"))  # Replace with your ngrok auth token
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url.public_url}")

    # Run FastAPI server
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    loop.run_until_complete(server.serve())

# Execute the server
if __name__ == "__main__":
    run_server()

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Drive already mounted at /content/drive
Public URL: https://99d3-34-86-49-74.ngrok-free.app


INFO:     Started server process [33616]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Connected to Neon PostgreSQL database
SentenceTransformer model loaded
Loaded existing FAISS CPU index from /content/drive/My Drive/lifesciences/faiss_index_retrained_batch_12.index
Fetching content for faiss_indices: [22, 6, 19]
INFO:     45.150.175.118:0 - "GET /query_content/?urs_name=Pharma_URS_Enhanced&section_name=Core%20Functionalities&k=3 HTTP/1.1" 200 OK
